## Recursive inference (multi-step) for BSCTRFM models

### use time series instead of SLDB arrays for easier and more efficient timestamp management

In [1]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# uncomment the following line for compatibility with TensorFlow 1.15 (on GCP)
# import tensorflow.compat.v1 as tf
# uncomment the following line for TensorFlow 2.X (local execution)
import tensorflow as tf

# forecast model was saved in TensorFlow 1.15
# but, in order to make predictions locally, has to be loaded with TensorFlow 2
from tensorflow.saved_model import load

In [2]:
# symmetric mean absolute percentage error
def symmetric_mean_absolute_percentage_error(targets, predictions):
    '''
    predictions: a list with the predicted values
    targets: a list with the actual values
    '''
    import numpy as np
    # lists to NumPy arrays
    targets, predictions = np.array(targets), np.array(predictions)
    # verify predictions and targets have the same shape
    if predictions.shape == targets.shape:
            return(np.sum(2*np.abs(predictions - targets) /
                          (np.abs(targets) + np.abs(predictions)))/predictions.shape[0])

In [3]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [4]:
# converts a set of tensors to a feature dict to a serialized example to pass it
# to the prediction function of the saved model 
def input_tensors_to_serialized_example(encoder_input_float_tensor,
                                        decoder_input_float_tensor,
                                        id_float_tensor):
    # first, pass the float tensors to NumPy array, then flatten them
    encoder_input_flat_array = encoder_input_float_tensor.numpy().flatten()
    decoder_input_flat_array = decoder_input_float_tensor.numpy().flatten()
    id_flat_array = id_float_tensor.numpy().flatten()
    
    # second, build the protobuffer example
    example = tf.train.Example(
        # features within the example
        features=tf.train.Features(
            # feature definition
            feature={
                'encoder_input': _float_feature_from_list_of_values(encoder_input_flat_array),
                'decoder_input': _float_feature_from_list_of_values(decoder_input_flat_array),
                'id': _float_feature_from_list_of_values(id_flat_array)
            }
        )
    )    
    # third, serialize the example dictionary to a string
    serialized_example = example.SerializeToString()
    # fourth, wrap the serialized example as a NumPy-string array
    numpy_example = np.array(serialized_example, dtype='S')
    # fifth, wrap the NumPy-string array as a string tensor
    serialized_example = tf.convert_to_tensor(numpy_example)

    return serialized_example

In [5]:
PROJECT_ROOT = '/home/developer/gcp/cbidmltsf'

In [6]:
# during batch prediction, the model identifier is obtained via Abseil Flags
# remember this notebook is based on local execution,
# therefore model directory must be downloaded from GS before running the notebook
model_id = 'BSCTRFM_TPU_050'

In [19]:
# during batch prediction, the execution identifier is obtained via Abseil Flags
execution = 0

In [8]:
# during batch prediction, the SLDB identifier is obtained via Abseil Flags
# THE SLDB FOR INFERENCE MUST BE THE SAME USED FOR TRAINING! (THE ONE SETUP IN THE CONFIGURATION FILE)
sldb_id = 'LD2011-2014_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX'

In [9]:
# during batch prediction, the dataset name is obtained via Abseil Flags
dataset = 'test'

In [10]:
# define a forecast window to guide the iterative prediction process
# start with a hourly, day-ahead process
forecast_window = 24

In [11]:
# ADD AN INFERENCE IDENTIFIER, BECAUSE FOR TRANSFORMER-BASED MODELS, DIFFERENT INFERENCES
# CAN BE PRODUCED FROM A SINGLE SAVED MODEL (USUALLY TO PRODUCE DIFFERENT FORECAST WINDOWS)
# during batch prediction, the inference identifier should be obtained via Abseil Flags
inference = '{:03d}'.format(forecast_window)

In [12]:
# build a path to the SLDB json file
data_dir = '{}/{}/{}'.format(PROJECT_ROOT, 'sldbs', sldb_id)

# then get the ts_identifier from the json file in the sldb directory
sldb_json_file = '{}/sldb.json'.format(data_dir)

In [13]:
# open the json file
with open(sldb_json_file, 'r') as inputfile:
    sldb_dict = json.load(inputfile)

In [14]:
# and get the time series identifier
ts_identifier = sldb_dict['ts']
ts_identifier

'LD2011-2014_SEPARATED_FULL'

In [15]:
# get the SLDB parameters for the forecasting model
config_json_file = '{}/{}/{}.json'.format(PROJECT_ROOT,
                                          'parameters',
                                          model_id)

# recover the sldb dictionary from the json file in parameters/
with open(config_json_file, 'r') as inputfile:
    configuration = json.load(inputfile)

In [16]:
m = sldb_dict['embedding']['hourly']
m

168

In [17]:
t = sldb_dict['no_targets']
t

168

In [20]:
# verify the values of the variables for batch inference
model_id, execution, dataset, inference

('BSCTRFM_TPU_050', 0, 'test', '024')

In [21]:
# build a list of saved models, given the parameters in the previous cell
saved_models = os.listdir('{}/models/{}_{:02d}/export/exporter'.format(PROJECT_ROOT, model_id, execution))

In [22]:
saved_models.sort()
saved_models

['1634918778',
 '1634919073',
 '1634919384',
 '1634919663',
 '1634919941',
 '1634920306',
 '1635256002',
 '1635256290',
 '1635256570',
 '1635256852',
 '1635257138',
 '1635257496',
 '1635257814',
 '1635258118',
 '1635258400',
 '1635258686',
 '1635258965',
 '1635259247',
 '1635259538',
 '1635390669']

In [23]:
# get the identifier for saved models at 1 epoch, executions 1 to 10

In [24]:
executions = list(np.arange(1, 10 + 1))
executions

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [27]:
# for each execution, get the first saved model id (at epoch 1)
saved_models = list()

for execution in executions:
    saved_models_list = os.listdir('{}/models/{}_{:02d}/export/exporter'.format(PROJECT_ROOT, model_id, execution))
    saved_models_list.sort()
    saved_models.append(saved_models_list[0])

In [28]:
for execution, saved_model in zip(executions, saved_models):
    print(execution, saved_model)

1 1635388035
2 1635388349
3 1635388634
4 1635388907
5 1635389193
6 1635389473
7 1635389773
8 1635390064
9 1635390345
10 1635390628


In [ ]:
# and we are ready to compute predictions over first saved models (at epoch 1)

In [29]:
encoder_input_columns = [
    'kw_scaled',
    'sin_hours_from_start',
    'cos_hours_from_start',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    # 'sin_day_month',
    # 'cos_day_month',
    # 'sin_day_year',
    # 'cos_day_year'
]

In [30]:
decoder_input_columns = encoder_input_columns

In [31]:
id_columns = ['token_id']

In [32]:
# now get the time series for the test dataset (unseen data)

In [33]:
# build the time series directory
time_series_folder = '{}/test'.format(data_dir)
time_series_folder

'/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX/test'

In [34]:
time_series_list = os.listdir(time_series_folder)
time_series_list.sort()

In [35]:
len(time_series_list)

369

In [36]:
# 369 customers from 'MT_001' to 'MT_370'
# (with the exception of 'MT_223', which has no lectures on the selected interval)

In [37]:
# individual test dataset for each customer
# explore any of them

In [38]:
test_df = pd.read_pickle(
    '/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX/test/MT_131.pkl'
)

In [39]:
test_df

,date,token_id,kw_scaled,sin_hours_from_start,cos_hours_from_start,sin_hour_day,cos_hour_day,sin_day_week,cos_day_week
3110221,2014-08-18 01:00:00,131,0.262887,-9.748783e-02,0.995237,0.258819,0.965926,0.000000,1.00000
3110222,2014-08-18 02:00:00,131,0.206186,-9.729425e-02,0.995256,0.500000,0.866025,0.000000,1.00000
3110223,2014-08-18 03:00:00,131,0.201031,-9.710066e-02,0.995275,0.707107,0.707107,0.000000,1.00000
3110224,2014-08-18 04:00:00,131,0.226804,-9.690707e-02,0.995293,0.866025,0.500000,0.000000,1.00000
3110225,2014-08-18 05:00:00,131,0.283505,-9.671348e-02,0.995312,0.965926,0.258819,0.000000,1.00000
...,...,...,...,...,...,...,...,...,...
3110719,2014-09-07 19:00:00,131,0.195876,-7.780311e-04,1.000000,-0.965926,0.258819,-0.781831,0.62349
3110720,2014-09-07 20:00:00,131,0.195876,-5.835234e-04,1.000000,-0.866025,0.500000,-0.781831,0.62349
3110721,2014-09-07 21:00:00,131,0.242268,-3.890156e-04,1.000000,-0.707107,0.707107,-0.781831,0.62349
3110722,2014-09-07 22:00:00,131,0.262887,-1.945078e-04,1.000000,-0.500000,0.866025,-0.781831,0.62349


In [40]:
# single time series from '2014-08-18 01:00:00' to '2014-09-07 23:00:00'

In [41]:
# from the above time series, a total of 168 sub-series or test examples are built, with 336 lectures each:

In [42]:
# sub-series 0 (first sub-series):
# encoder_input: '2014-08-18 01:00:00' to '2014-08-25 00:00:00'
# decoder_input: '2014-08-25 00:00:00' to '2014-08-31 23:00:00'
# decoder_output: '2014-08-25 01:00:00' to '2014-09-01 00:00:00'
# forecast window is built (iterative inference) from the last element in the decoder output (first unseen)
# from '2014-09-01 00:00:00' to '2014-09-01 23:00:00'
test_df[0:336]

,date,token_id,kw_scaled,sin_hours_from_start,cos_hours_from_start,sin_hour_day,cos_hour_day,sin_day_week,cos_day_week
3110221,2014-08-18 01:00:00,131,0.262887,-0.097488,0.995237,0.258819,0.965926,0.000000,1.00000
3110222,2014-08-18 02:00:00,131,0.206186,-0.097294,0.995256,0.500000,0.866025,0.000000,1.00000
3110223,2014-08-18 03:00:00,131,0.201031,-0.097101,0.995275,0.707107,0.707107,0.000000,1.00000
3110224,2014-08-18 04:00:00,131,0.226804,-0.096907,0.995293,0.866025,0.500000,0.000000,1.00000
3110225,2014-08-18 05:00:00,131,0.283505,-0.096713,0.995312,0.965926,0.258819,0.000000,1.00000
...,...,...,...,...,...,...,...,...,...
3110552,2014-08-31 20:00:00,131,0.216495,-0.033255,0.999447,-0.866025,0.500000,-0.781831,0.62349
3110553,2014-08-31 21:00:00,131,0.201031,-0.033060,0.999453,-0.707107,0.707107,-0.781831,0.62349
3110554,2014-08-31 22:00:00,131,0.206186,-0.032866,0.999460,-0.500000,0.866025,-0.781831,0.62349
3110555,2014-08-31 23:00:00,131,0.226804,-0.032671,0.999466,-0.258819,0.965926,-0.781831,0.62349


In [43]:
# sub-series 144 (145-th sub-series):
# encoder_input: '2014-08-24 01:00:00' to '2014-08-31 00:00:00'
# decoder_input: '2014-08-31 00:00:00' to '2014-09-07 23:00:00'
# decoder_output: '2014-08-31 01:00:00' to '2014-09-07 00:00:00'
# forecast window is built (iterative inference) from the last element in the decoder output (first unseen)
# from '2014-09-07 00:00:00' to '2014-09-07 23:00:00'
test_df[0+144:336+144]

,date,token_id,kw_scaled,sin_hours_from_start,cos_hours_from_start,sin_hour_day,cos_hour_day,sin_day_week,cos_day_week
3110365,2014-08-24 01:00:00,131,0.077320,-0.069578,0.997577,0.258819,0.965926,-0.781831,0.623490
3110366,2014-08-24 02:00:00,131,0.072165,-0.069383,0.997590,0.500000,0.866025,-0.781831,0.623490
3110367,2014-08-24 03:00:00,131,0.134021,-0.069189,0.997604,0.707107,0.707107,-0.781831,0.623490
3110368,2014-08-24 04:00:00,131,0.185567,-0.068995,0.997617,0.866025,0.500000,-0.781831,0.623490
3110369,2014-08-24 05:00:00,131,0.237113,-0.068801,0.997630,0.965926,0.258819,-0.781831,0.623490
...,...,...,...,...,...,...,...,...,...
3110696,2014-09-06 20:00:00,131,0.695876,-0.005252,0.999986,-0.866025,0.500000,-0.974928,-0.222521
3110697,2014-09-06 21:00:00,131,0.268041,-0.005057,0.999987,-0.707107,0.707107,-0.974928,-0.222521
3110698,2014-09-06 22:00:00,131,0.072165,-0.004863,0.999988,-0.500000,0.866025,-0.974928,-0.222521
3110699,2014-09-06 23:00:00,131,0.072165,-0.004668,0.999989,-0.258819,0.965926,-0.974928,-0.222521


In [44]:
# starting indexes for predictions can be defined in two ways:

In [45]:
# produce all prediction sets

# to produce 168 different hourly-day-ahead prediction sets,

# starting at '2014-09-01 00:00:00', '2014-09-01 01:00:00', '2014-09-01 02:00:00'...
# up to '2014-09-07 00:00:00'

# only 7 of this predictions set are required for benchmarking
# but analyzing the total 168 sets would be valuable for model performance tunning

In [46]:
# produce only the predictions set required for benchmarking

# to produce 7 hourly-day-ahead prediction sets to cover the test interval

# starting at '2014-09-01 00:00:00', '2014-09-02 00:00:00', '2014-09-03 00:00:00', '2014-09-04 00:00:00',
#             '2014-09-05 00:00:00', '2014-09-06 00:00:00', '2014-09-07 00:00:00'

In [47]:
# get and test the starting indexes list to produce 7 hourly-day-ahead prediction sets for benchmarking

dataset_row_indexes_list = [0, 24, 48, 72, 96, 120, 144]

for start_index in dataset_row_indexes_list:
    # for each row, print the dates of the last element in the decoder output
    # and the end of the generated forecast window
    print(test_df.iloc[start_index+335]['date'], test_df.iloc[start_index+335+23]['date'])

2014-09-01 00:00:00 2014-09-01 23:00:00
2014-09-02 00:00:00 2014-09-02 23:00:00
2014-09-03 00:00:00 2014-09-03 23:00:00
2014-09-04 00:00:00 2014-09-04 23:00:00
2014-09-05 00:00:00 2014-09-05 23:00:00
2014-09-06 00:00:00 2014-09-06 23:00:00
2014-09-07 00:00:00 2014-09-07 23:00:00


In [48]:
# use a dictionary to remain the code consistent with the SLDB building process
# most of the times, only ts['test'] will be used for inference
# however, ts['eval'] might also be used, as it have not really been seen by training process
# (no modification in model training resulted from evaluation stage)

### rename the time series dictionary to ts_test, use the customer_id as key

In [49]:
ts_test = dict()

In [50]:
# a dictionary to store predictions detail dataframe per customer id
predictions_detail = dict()

In [51]:
start, end = 1, 370

In [52]:
all_customer_ids = ['MT_{:03d}'.format(token_id) for token_id in np.arange(start, end + 1)]

In [53]:
customer_ids = list(set(all_customer_ids).difference(set(['MT_223'])))

In [54]:
customer_ids.sort()

In [55]:
len(customer_ids)

369

## skip the following code to avoid inference process

In [54]:
# a second inference identifier to run more than one inference
# on the same combination model_id, execution, inference;
# to produce different inference processes because predict_fn is stochastic
event = 0

for customer_id in customer_ids:
    
    # read the time series for the current customer
    ts_test[customer_id] = pd.read_pickle('{}/test/{}.pkl'.format(data_dir, customer_id))

    # for consistency, rename the column 'date' as 'timestamp'
    ts_test[customer_id] = ts_test[customer_id].rename(columns={"date": "timestamp"})
    # set the column 'timestamp' as index
    ts_test[customer_id] = ts_test[customer_id].set_index('timestamp')

    # report start and end timestamp for the loaded time series
    print('Loaded test time series for {}, which spans from {} to {}'.\
         format(customer_id,
                str(ts_test[customer_id].index[:1][0]),
                str(ts_test[customer_id].index[-1:][0])))

    scaler_type = 'min_max'
    
    if scaler_type == 'min_max':
        # build a path to the scaler of the time series
        scaler_path = '{}/scalers/min_max_{}.save'.format(data_dir, customer_id)
        # and load it
        scaler = joblib.load(scaler_path)


    '''
    if scaler_type == 'standard':
        # build a path to the scaler of the time series
        scaler_path = '{}/scalers/standard_{}.save'.format(data_dir, customer_id)
        # and load it
        scaler = joblib.load(scaler_path)
    '''    
        
        
    # pass the saved model identifier (it determines the number of training epochs)
    # and avoid using the latest saved model by default
    for saved_model_id in saved_models[6:]:

        # a columns list for the predictions dataframe
        pred_df_columns = ['model_id',
                           'execution',
                           'dataset',
                           'inference',
                           'customer_id',
                           'string_timestamps',
                           'predictions',
                           'targets']

        # build the predictions dataframe as a key-value pair of the dictionary
        predictions_detail[customer_id] = pd.DataFrame(columns=pred_df_columns)

        # use model identifier and execution number to build the model directory string
        model_dir = '{}_{:02d}'.format(model_id, execution)

        # get the path to the saved model main directory
        saved_model_path = '{}/{}/{}/export/exporter'.format(PROJECT_ROOT,
                                                             'models',
                                                             model_dir)

        # get all the files in the saved model path, to find the most recent one
        # all_files = os.listdir(saved_model_path)
        # get the path to the most recent saved model
        # latest_saved_model_id = sorted(all_files)[-1]

        # build the full path for the latest saved model dir
        export_dir = '{}/{}'.format(saved_model_path, saved_model_id)
        print ('Exported model path is {}'.format(export_dir))

        # load the saved model and the prediction function
        imported = load(export_dir=export_dir, tags='serve')
        predict_fn = imported.signatures["serving_default"]

        # iterate on a set of valid rows of the test dataset
        starting_point = 0 # based on the inference dataset
        span = 1 + 6*24 # number of days in the test dataset, expressed in hours
        
        # MAKE THE PREDICTION SETS FOR BENCHMARKING ONLY! 
        # dataset_row_indexes_list = starting_point + np.arange(span)
        dataset_row_indexes_list = [0, 24, 48, 72, 96, 120, 144]
        
        for start_index in dataset_row_indexes_list:

            # define first prediction interval with start- and end-index
            # given the interval time_series[start_index:end_index]
            # the conditioning range is the union of the encoder-input and the decoder-input
            # and the prediction range is only the last lecture in the interval,
            # by means of a recursive inference process
            # on each step the last prediction is added to the decoder input
            # and the prediction range grows one step into the future

            # get the end-index of this recursive inference interval
            end_index = start_index + (m + t)

            # initialize a list to store recurrent predictions for this interval
            predictions_list = list()

            for i in np.arange(forecast_window):

                # build the inference interval as a sub-series of the dataset
                # the following slicing operation is equivalent to:
                # sub_series = ts_test[customer_id].iloc[start_index + i : end_index + i]
                sub_series = ts_test[customer_id][start_index + i : end_index + i]

                # important: build sources as copies of the sub-series (and therefore of the global time series)
                # to avoid overwriting the original dataset

                # the encoder input source
                encoder_input = sub_series[encoder_input_columns][:m].copy()

                # the decoder input source
                # decoder_input = sub_series[m-1:-1].copy()
                decoder_input = sub_series[decoder_input_columns][m-1:m-1+t].copy()

                # the id (integer) for the customer
                id_input = sub_series[id_columns][:1].copy()

                # on first step (i=0), the decoder input carries only true values
                # and the predictions list is empty
                # on subsequent steps, the decoder input includes all previous predictions
                # (stored in the predictions list)
                if i > 0:
                    decoder_input['kw_scaled'][-i:] = predictions_list

                # the target source, for metrics calculation
                # the first part of the sub-series is the encoder input, and
                # the second part of the sub-series is the target (only the variable column!)
                target = sub_series['kw_scaled'][m:].copy()

                # build source tensors from the sub-series    
                encoder_input_tensor = tf.expand_dims(encoder_input, axis=0)
                decoder_input_tensor = tf.expand_dims(decoder_input, axis=0)
                id_tensor = tf.expand_dims(id_input, axis=0)

                # make input example for the prediction function
                input_example = input_tensors_to_serialized_example(encoder_input_tensor,
                                                                    decoder_input_tensor,
                                                                    id_tensor)

                # get the output of the prediction function as a dictionary
                predict_output_dict = predict_fn(input_example)

                # get the prediction output tensor
                predict_output_tensor = predict_output_dict['forecast']

                # get the most recent prediction
                most_recent_prediction = predict_output_tensor[0, :, 0].numpy()[-1]

                # append the most recent prediction timestep to the predictions list
                predictions_list.append(most_recent_prediction)

                # pass the predictions list to an array
                # current_predictions_array = np.array(predictions_list).reshape(-1, 1)
                # get the targets vector to be compared with the current predictions array
                # current_targets = np.array(target[-i-1:]).reshape(-1, 1)

                # calculate SMAPE on the rescaled variable
                # rescaled_predictions = min_max_scaler.inverse_transform(current_predictions_array)
                # rescaled_targets = min_max_scaler.inverse_transform(current_targets)

                # current_smape = symmetric_mean_absolute_percentage_error(
                #     rescaled_targets, rescaled_predictions)
                # print('On row {}, SMAPE for the first {} rescaled prediction(s) is {}'.format(start_index,
                #                                                                               i + 1,
                #                                                                               current_smape))        

            # iterative predictions over the forecast window reside in predictions_list
            # convert list to array, then expand feature dimension with value 1
            predicted_values = np.array(predictions_list).reshape(-1, 1)

            # inverse-scale predictions
            rescaled_predicted_values = scaler.inverse_transform(predicted_values)

            # and the true values remain in the prediction tensor, pass them to a NumPy array
            # for the true values array, expand feature dimension with value 1
            true_values = np.array(target[-i-1:]).reshape(-1, 1)

            # inverse-scale true values
            rescaled_true_values = scaler.inverse_transform(true_values)

            # a temporary dataframe built from the data in the current row
            df = pd.DataFrame(columns=pred_df_columns)
            df['model_id'] = [model_id]
            df['execution'] = [execution]
            df['dataset'] = [dataset]
            df['inference'] = [inference]
            df['customer_id'] = [customer_id]
            df['string_timestamps']= [pd.to_datetime(target.index[-i-1:]).astype(str).tolist()]
            df['predictions'] = [np.squeeze(rescaled_predicted_values).tolist()]
            df['targets'] = [np.squeeze(rescaled_true_values).tolist()]

            # calculate mean absolute error and normalized deviation
            mae = mean_absolute_error(rescaled_true_values, rescaled_predicted_values)
            df['mae'] = mae

            true_values_average = np.mean(rescaled_true_values)
            df['nd'] = mae/true_values_average

            # calculate root mean squared error and normalized root mean squared error
            rmse = sqrt(mean_squared_error(rescaled_true_values, rescaled_predicted_values))
            df['rmse'] = rmse
            df['nrmse'] = rmse/true_values_average

            df['smape'] = symmetric_mean_absolute_percentage_error(rescaled_true_values,
                                                                   rescaled_predicted_values)

            # append the temporary dataframe to the predictions detail dataframe
            predictions_detail[customer_id] = pd.concat([predictions_detail[customer_id], df])


        # reset the index of final dataframe, once all of its rows (dataset) have been processed
        predictions_detail[customer_id] = predictions_detail[customer_id].reset_index(drop=True)


        # hard-wired switcher to persist dataframe
        persist_detail = True
        
        if persist_detail:
            # build a path to persist the dataframe to database/predictions_detail/
            detail_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}_{}_{:02d}.pkl'.format(
                PROJECT_ROOT,
                'database',
                'predictions_detail',
                model_id,
                execution,
                saved_model_id,
                # for electricity dataset, replace dataset with customer_id
                customer_id,
                inference,
                event)

            # persist the Pandas dataframe to database/predictions_detail/
            predictions_detail[customer_id].to_pickle(detail_pickle_path)
            print(
                'Persisted predictions detail of {}_{:02d}_{}_{}_{}_{:02d}'.format(model_id,
                                                                                   execution,
                                                                                   saved_model_id,
                                                                                   customer_id,
                                                                                   inference,
                                                                                   event)
            )

Loaded test time series for MT_001, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256002
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256002_MT_001_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256290
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256290_MT_001_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256570
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256570_MT_001_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256852
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256852_MT_001_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635257138
Persisted predictions detail of BSCTRFM_TPU_050_00_1635257138_MT_001_0

Persisted predictions detail of BSCTRFM_TPU_050_00_1635257138_MT_004_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635257496
Persisted predictions detail of BSCTRFM_TPU_050_00_1635257496_MT_004_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635257814
Persisted predictions detail of BSCTRFM_TPU_050_00_1635257814_MT_004_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258118
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258118_MT_004_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258400
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258400_MT_004_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258686
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258686_MT_004_024_00
Exported model

Persisted predictions detail of BSCTRFM_TPU_050_00_1635258686_MT_007_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258965
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258965_MT_007_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635259247
Persisted predictions detail of BSCTRFM_TPU_050_00_1635259247_MT_007_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635259538
Persisted predictions detail of BSCTRFM_TPU_050_00_1635259538_MT_007_024_00
Loaded test time series for MT_008, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256002
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256002_MT_008_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/16352

Persisted predictions detail of BSCTRFM_TPU_050_00_1635256002_MT_011_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256290
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256290_MT_011_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256570
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256570_MT_011_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256852
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256852_MT_011_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635257138
Persisted predictions detail of BSCTRFM_TPU_050_00_1635257138_MT_011_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635257496
Persisted predictions detail of BSCTRFM_TPU_050_00_1635257496_MT_011_024_00
Exported model

Persisted predictions detail of BSCTRFM_TPU_050_00_1635257496_MT_014_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635257814
Persisted predictions detail of BSCTRFM_TPU_050_00_1635257814_MT_014_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258118
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258118_MT_014_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258400
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258400_MT_014_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258686
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258686_MT_014_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258965
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258965_MT_014_024_00
Exported model

...

Persisted predictions detail of BSCTRFM_TPU_050_00_1635256290_MT_355_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256570
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256570_MT_355_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256852
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256852_MT_355_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635257138
Persisted predictions detail of BSCTRFM_TPU_050_00_1635257138_MT_355_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635257496
Persisted predictions detail of BSCTRFM_TPU_050_00_1635257496_MT_355_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635257814
Persisted predictions detail of BSCTRFM_TPU_050_00_1635257814_MT_355_024_00
Exported model

Persisted predictions detail of BSCTRFM_TPU_050_00_1635257814_MT_358_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258118
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258118_MT_358_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258400
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258400_MT_358_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258686
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258686_MT_358_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258965
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258965_MT_358_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635259247
Persisted predictions detail of BSCTRFM_TPU_050_00_1635259247_MT_358_024_00
Exported model

Persisted predictions detail of BSCTRFM_TPU_050_00_1635259247_MT_361_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635259538
Persisted predictions detail of BSCTRFM_TPU_050_00_1635259538_MT_361_024_00
Loaded test time series for MT_362, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256002
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256002_MT_362_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256290
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256290_MT_362_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256570
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256570_MT_362_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/16352

Persisted predictions detail of BSCTRFM_TPU_050_00_1635256570_MT_365_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635256852
Persisted predictions detail of BSCTRFM_TPU_050_00_1635256852_MT_365_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635257138
Persisted predictions detail of BSCTRFM_TPU_050_00_1635257138_MT_365_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635257496
Persisted predictions detail of BSCTRFM_TPU_050_00_1635257496_MT_365_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635257814
Persisted predictions detail of BSCTRFM_TPU_050_00_1635257814_MT_365_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258118
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258118_MT_365_024_00
Exported model

Persisted predictions detail of BSCTRFM_TPU_050_00_1635258118_MT_368_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258400
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258400_MT_368_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258686
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258686_MT_368_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635258965
Persisted predictions detail of BSCTRFM_TPU_050_00_1635258965_MT_368_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635259247
Persisted predictions detail of BSCTRFM_TPU_050_00_1635259247_MT_368_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_050_00/export/exporter/1635259538
Persisted predictions detail of BSCTRFM_TPU_050_00_1635259538_MT_368_024_00
Loaded test ti